# STOR 320: Introduction to Data Science
## Lab 9

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

c:\Users\conor\AppData\Local\Programs\Python\Python311\Lib\site-packages\matplotlib\projections\__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


1. We use the following code to generate two columns of features and the target values. Based on the code below, what is the true linear model between `Target` and two features?

In [2]:
np.random.seed(0)

# Generate feature matrix X (100 samples, 2 features)
X = np.random.rand(100, 2)

# Define true coefficients for generating y
true_beta = np.array([5, 2, -3])  # Intercept, beta_1, beta_2

# Generate y with some added noise
y = true_beta[0] + X @ true_beta[1:] + np.random.normal(0, 0.5, X.shape[0])

# Combine X and y into a pandas DataFrame
data = pd.DataFrame(np.column_stack((X, y)), columns=['Feature_1', 'Feature_2', 'Target'])
data

,Feature_1,Feature_2,Target
0,0.548814,0.715189,4.515377
1,0.602763,0.544883,4.030911
2,0.423655,0.645894,3.335893
3,0.437587,0.891773,2.980945
4,0.963663,0.383442,5.527985
...,...,...,...
95,0.398221,0.209844,4.879017
96,0.186193,0.944372,2.610245
97,0.739551,0.490459,4.848061
98,0.227415,0.254356,5.037529


$$Target=5+2⋅Feature_1−3⋅Feature_2+noise$$

2. Based on `data` table, separate X (features) and y (target) from the data. In other words, create a `100*2` numpy matrix for X and a numpy vector for y.

In [4]:
X = data[['Feature_1', 'Feature_2']].values
y = data['Target'].values
X, y 

(array([[0.5488135 , 0.71518937],
        [0.60276338, 0.54488318],
        [0.4236548 , 0.64589411],
        [0.43758721, 0.891773  ],
        [0.96366276, 0.38344152],
        [0.79172504, 0.52889492],
        [0.56804456, 0.92559664],
        [0.07103606, 0.0871293 ],
        [0.0202184 , 0.83261985],
        [0.77815675, 0.87001215],
        [0.97861834, 0.79915856],
        [0.46147936, 0.78052918],
        [0.11827443, 0.63992102],
        [0.14335329, 0.94466892],
        [0.52184832, 0.41466194],
        [0.26455561, 0.77423369],
        [0.45615033, 0.56843395],
        [0.0187898 , 0.6176355 ],
        [0.61209572, 0.616934  ],
        [0.94374808, 0.6818203 ],
        [0.3595079 , 0.43703195],
        [0.6976312 , 0.06022547],
        [0.66676672, 0.67063787],
        [0.21038256, 0.1289263 ],
        [0.31542835, 0.36371077],
        [0.57019677, 0.43860151],
        [0.98837384, 0.10204481],
        [0.20887676, 0.16130952],
        [0.65310833, 0.2532916 ],
        [0.466

3. Add a column of ones to X to account for the intercept term in the coefficient vector.

In [6]:
X_with_intercept = np.column_stack((np.ones(X.shape[0]), X))
X_with_intercept


array([[1.        , 0.5488135 , 0.71518937],
       [1.        , 0.60276338, 0.54488318],
       [1.        , 0.4236548 , 0.64589411],
       [1.        , 0.43758721, 0.891773  ],
       [1.        , 0.96366276, 0.38344152],
       [1.        , 0.79172504, 0.52889492],
       [1.        , 0.56804456, 0.92559664],
       [1.        , 0.07103606, 0.0871293 ],
       [1.        , 0.0202184 , 0.83261985],
       [1.        , 0.77815675, 0.87001215],
       [1.        , 0.97861834, 0.79915856],
       [1.        , 0.46147936, 0.78052918],
       [1.        , 0.11827443, 0.63992102],
       [1.        , 0.14335329, 0.94466892],
       [1.        , 0.52184832, 0.41466194],
       [1.        , 0.26455561, 0.77423369],
       [1.        , 0.45615033, 0.56843395],
       [1.        , 0.0187898 , 0.6176355 ],
       [1.        , 0.61209572, 0.616934  ],
       [1.        , 0.94374808, 0.6818203 ],
       [1.        , 0.3595079 , 0.43703195],
       [1.        , 0.6976312 , 0.06022547],
       [1.

4. Calculate the estimation of parameter $\beta$ manually using NumPy's matrix operations.
- Hint: You can refer to the lecture notes to find the closed-form solution for the regression coefficients
- Hint: You can use `np.linalg.inv` to calculate the inverse of a matrix.

In [7]:
beta_manual = np.linalg.inv(X_with_intercept.T @ X_with_intercept) @ X_with_intercept.T @ y
beta_manual

array([ 5.05725163,  1.78685022, -2.98521247])

5. Compute $R$ squared manually. You can refer to the lecture notes to see the definitino of R-squared.

In [8]:
y_pred = X_with_intercept @ beta_manual

SS_res = np.sum((y - y_pred) ** 2)

SS_tot = np.sum((y - y.mean()) ** 2)
R_squared_manual = 1 - (SS_res / SS_tot)
R_squared_manual


0.8308337212672313

6. **Comparison with statsmodels**

We fit a linear regression model using `statsmodels`, then print and compare both the manually calculated coefficients and R-squared values with those from statsmodels. Are they the same?

In [9]:
import statsmodels.api as sm

X_sm = sm.add_constant(X)
model = sm.OLS(y, X_sm).fit()

beta_statsmodels = model.params
R_squared_statsmodels = model.rsquared

# Display the results
print("Manual Coefficients:", beta_manual)
print("Statsmodels Coefficients:", beta_statsmodels)
print("Manual R-squared:", R_squared_manual)
print("Statsmodels R-squared:", R_squared_statsmodels)


Manual Coefficients: [ 5.05725163  1.78685022 -2.98521247]
Statsmodels Coefficients: [ 5.05725163  1.78685022 -2.98521247]
Manual R-squared: 0.8308337212672313
Statsmodels R-squared: 0.8308337212672314
